## Document Loading

Load a blog post on agents.

In [ ]:
! pip install pdf2image

Load academic papers -

In [7]:
import arxiv
from langchain_community.chat_models import ChatAnthropic
from langchain_community.document_loaders import ArxivLoader, UnstructuredPDFLoader

# Load a paper to use
paper = next(arxiv.Search(query="Visual Instruction Tuning").results())
paper.download_pdf(filename="downloaded-paper.pdf")
loader = UnstructuredPDFLoader("downloaded-paper.pdf")
doc = loader.load()[0]

Or try loading blog posts -

In [3]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
text = loader.load()

## Run template

In `server.py`, set -
```
add_routes(app, chain_rag_conv, path="/summarize-anthropic")
```

In [15]:
from langserve.client import RemoteRunnable

summarization = RemoteRunnable("http://localhost:8000/summarize-anthropic")
summarization.invoke({"text": doc.page_content})

" Here is my attempt to complete the requested tasks:\n\n<kindergarten_abstract>\nThe scientists made a robot friend that can see pictures and talk about them. First they let a big talking robot make up fun conversations about pictures. Then they taught their robot friend to have conversations too. Their robot friend got pretty good at seeing pictures and talking about them!\n</kindergarten_abstract>\n\n<moosewood_methods>\nVisual Instruction Tuning\nRecipe adapted from Haotian Liu et al.\n\nIngredients:\n- 1 large language model (we used GPT-4)\n- 158,000 images with text descriptions \n- 1 vision model (we used CLIP)\n- 1 large multimodal model (we used LLaMA)\n\nInstructions:\n1. Have the GPT-4 model make up conversations, long descriptions, and complex stories about the images, using only the text descriptions as hints. \n2. Connect the vision model and language model together into one big multimodal model. This will be your robot friend!\n3. Show the robot friend examples of the c